In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2

In [5]:
# Define data directories
train_dir = 'C:/Users/ashiv/Documents/action state or stop state/frames/train'
batch_size = 32
image_size = (224, 224)

In [6]:
# Create a data generator with data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  # Binary classification: 'in action' and 'stop'
    shuffle=True)

Found 582 images belonging to 2 classes.


In [7]:
# Create a simple MobileNetV2-based model
base_model = MobileNetV2(input_shape=(224, 224, 3),
                         include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model weights

model = keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

In [8]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
# Train the model
epochs = 10
model.fit(train_generator, epochs=epochs)

Epoch 1/10
19/19 [==============================] - 23s 1s/step - loss: 0.6789 - accuracy: 0.6100
Epoch 2/10
19/19 [==============================] - 21s 1s/step - loss: 0.4994 - accuracy: 0.7595
Epoch 3/10
19/19 [==============================] - 21s 1s/step - loss: 0.4113 - accuracy: 0.8557
Epoch 4/10
19/19 [==============================] - 21s 1s/step - loss: 0.3478 - accuracy: 0.8763
Epoch 5/10
19/19 [==============================] - 21s 1s/step - loss: 0.3086 - accuracy: 0.9038
Epoch 6/10
19/19 [==============================] - 21s 1s/step - loss: 0.2846 - accuracy: 0.9055
Epoch 7/10
19/19 [==============================] - 21s 1s/step - loss: 0.2776 - accuracy: 0.9089
Epoch 8/10
19/19 [==============================] - 21s 1s/step - loss: 0.2453 - accuracy: 0.9227
Epoch 9/10
19/19 [==============================] - 21s 1s/step - loss: 0.2372 - accuracy: 0.9192
Epoch 10/10
19/19 [==============================] - 22s 1s/step - loss: 0.2418 - accuracy: 0.9107


In [10]:
# Save the trained model
model.save('classification_model.h5')

In [12]:
# Convert the model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\ashiv\AppData\Local\Temp\tmpntusznnu\assets


INFO:tensorflow:Assets written to: C:\Users\ashiv\AppData\Local\Temp\tmpntusznnu\assets


In [13]:
# Save the TensorFlow Lite model
with open('classification_model.tflite', 'wb') as f:
    f.write(tflite_model)